In [1]:
import numpy as np
import torch
# add path
import sys
import os
sys.path.append('../')
from mfbox import load_model

In [2]:
# load data

# xLF input and output
x_xl = np.loadtxt('pre_N_xL-H_stitch_z0/train_input.txt')
y_h = np.loadtxt('pre_N_xL-H_stitch_z0/train_output.txt')




In [3]:
# predict y_lin

# load model
model = load_model('../models/pre_N_xLH_stitch_3sLin_z0/best_model.pth')

#normalize input
# bounds
bounds = np.loadtxt('pre_N_xL-H_stitch_z0/input_limits.txt')
# normalize the input data (only the first matched columns)
dimx_original = bounds.shape[0]
x_xl_norm = x_xl.copy()
x_xl_norm[:,:dimx_original] = (x_xl[:,:dimx_original] - bounds[:,0]) / (bounds[:,1] - bounds[:,0])

# predict
model.eval()

# to tensor
x_xl_norm = torch.tensor(x_xl_norm, dtype=torch.float32)
y_lin = model(x_xl_norm)


In [4]:
y_lin = y_lin.detach().numpy()

In [5]:
x = x_xl[:,:dimx_original]
y_l = x_xl[:,dimx_original:]

In [6]:
# concatenate y_l and y_lin in such way that y_l_1, y_lin_1, y_l_2, y_lin_2, ...
# y = np.zeros((y_lin.shape[0], 2*y_lin.shape[1]))
# y[:,::2] = y_l
# y[:,1::2] = y_lin

y = np.concatenate((y_l, y_lin), axis=1)
# y = y_l

In [7]:
# create dataset for LF-HF correlation
# input: x_h, y_l1[indices], y_l2[indices] (concatenate)
x_xllin = np.concatenate((x, y), axis=1)

# output: y_h 
y_h = y_h

In [8]:
x_xllin.shape

(27, 138)

In [9]:
# save the dataset
save_dir = 'pre_N_xL-H_stitch_3sNonlin_z0'
# mkdir if not exist

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

np.savetxt(save_dir + '/train_input.txt', x_xllin)
np.savetxt(save_dir + '/train_output.txt', y_h)